# Raman model in Pytorch 
This notebook as the purpose to transpose the CNN model created by Dario Bertazioli in the Pytorch librairie.

## Librairies
Here, we will find the librairies needed for this work. 

In [1]:
# --- Utils librairies ---
#Generics librairies
import os
import os.path
from os import path
import numpy as np
import copy
import pickle
from statistics import mean
import matplotlib.pyplot as plt

#Measure librairies
import time

#Dataset librairies
import pandas as pd

# --- DL librairies ---
#Pytorch librairies
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader

#Sklearn librairies
from sklearn.model_selection import LeaveOneGroupOut, train_test_split
from sklearn.metrics import accuracy_score

## Set up the GPU environment

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3,4";
gpus_list = [0, 1, 2, 3]

print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
4


## Data
In this part, we will find all the methods regarding the data (loading, data augmentation...). 

### Load data

In [3]:
df, labels, names = pd.read_pickle("../data/dataset_COVID_RAW.pkl")
df = pd.DataFrame(df)
labels = pd.Series(labels, name = 'label')
names = pd.Series(names, name = 'names')
df = pd.concat((df, labels, names), axis = 1)

In [4]:
X_set = df.drop(columns = ['label', "names"]).values
Y_set = df.label.values
groups = df.names.values

In [5]:
folds = list(LeaveOneGroupOut().split(X_set, Y_set, groups = groups))

### Data augmentation

In [6]:
def dataAugment(signal, betashift = 0.24039033704204857, slopeshift = 0.5640435054299953, multishift = 0.0013960388613510225):
    #baseline shift
    beta = np.random.random(size=(signal.shape[0],1))*2*betashift-betashift
    slope = np.random.random(size=(signal.shape[0],1))*2*slopeshift-slopeshift + 1
    #relative positions
    axis = np.array(range(signal.shape[1]))/float(signal.shape[1])
    #offset
    offset = slope*(axis) + beta - axis - slope/2. + 0.5

    #multiplicative coefficient
    multi = np.random.random(size=(signal.shape[0],1))*2*multishift-multishift + 1
    augmented_signal = multi*signal + offset

    return augmented_signal

### Creation of train, test and validation sets

In [7]:
class RamanDataset(Dataset):
    def __init__(self, X, Y):
        super(RamanDataset).__init__()
        x = torch.from_numpy(X)
        self.raman_spectra = x
        y = torch.from_numpy(Y)
        self.labels = y
        
    def __len__(self):
        return len(self.raman_spectra)
    
    def __getitem__(self, idx):
        spectrum = self.raman_spectra[idx]
        label = self.labels[idx]
        
        return spectrum, label

In [8]:
class Datasets(Dataset):
    def __init__(self, ramanDatasets):
        super(Datasets).__init__()
        self.datasets = ramanDatasets
        
    def __len__(self):
        return len(self.datasets)
    
    def __getitem__(self, idx):
        return self.datasets[idx]

In [9]:
def setsCreation(pathToFile, X_set, Y_set, folds):
    if not path.exists(pathToFile):
        train_set = []
        validation_set = []
        test_set = []
        X_train = []
        X_val = []
        X_test = []
        Y_train = []
        Y_val = []
        Y_test = []
        
        for i, (train_idx, test_idx) in enumerate(folds):
            X_train_tmp = X_set[train_idx]
            Y_train_tmp = Y_set[train_idx]
            
            X_test_tmp = X_set[test_idx]
            Y_test_tmp = Y_set[test_idx]
            
            X_train_tmp, X_val_tmp, Y_train_tmp, Y_val_tmp = train_test_split(X_train_tmp, Y_train_tmp, test_size = .1,
                                                                              stratify = Y_train_tmp)
            augment = 30
            augmented_data = []
            Y_list = copy.copy(Y_train_tmp)
            for i in range(augment):
                augmented_data.append(dataAugment(X_train_tmp))
            for i in range(augment-1):
                Y_list = np.concatenate((Y_list, Y_train_tmp), axis=0)
                
            X_train_tmp = np.vstack(augmented_data)
            Y_train_tmp = copy.copy(Y_list)
            train_set_tmp = RamanDataset(X_train_tmp, Y_train_tmp)
            train_set.append(train_set_tmp)
            val_set_tmp = RamanDataset(X_val_tmp, Y_val_tmp)
            validation_set.append(val_set_tmp)
            test_set_tmp = RamanDataset(X_test_tmp, Y_test_tmp)
            test_set.append(test_set_tmp)
        
        train_dataset = Datasets(train_set)
        validation_dataset = Datasets(validation_set)
        test_dataset = Datasets(test_set)
        training_settings = (train_dataset, validation_dataset, test_dataset)
        
        with open(pathToFile, "wb") as outf:
            pickle.dump(training_settings, outf)
        
    else:
        with open(pathToFile, "rb") as inf:
            train_dataset, validation_dataset, test_dataset = pickle.load(inf)
            
    return train_dataset, validation_dataset, test_dataset

In [10]:
train_dataset, validation_dataset, test_dataset = setsCreation("../train_settings/training_settings_cov_raw.pckl", X_set, Y_set, folds)

## Model

### Utility class and functions

In [11]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(1, 100, kernel_size=100,
                     stride=1, padding_mode='replicate'),
            nn.ReLU(),
            nn.BatchNorm1d(100, eps=0.001, momentum=0.99),
            nn.Conv1d(100, 102, kernel_size=5,
                     stride=2, padding_mode='replicate'),
            nn.ReLU(),
            nn.MaxPool1d(6, stride=3),
            nn.BatchNorm1d(102, eps=0.001, momentum=0.99),
            nn.Conv1d(102, 25, kernel_size=9,
                     stride=5, padding_mode='replicate'),
            nn.ReLU(),
            nn.MaxPool1d(3, stride=2)
        )
        
        self.dense_layers = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(325, 732),
            nn.LeakyReLU(),
            nn.Dropout(p=0.7000000000000001),
            nn.Linear(732, 152),
            nn.LeakyReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(152,189),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(189, 3),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = x.resize_(x.shape[0],  1, x.shape[1])
        x = self.cnn_layers(x)
        x = torch.flatten(x, 1)
        x = self.dense_layers(x)
        return x        

In [12]:
def createCNN(gpu_ids):
    model = ConvNet()
    model = model.double()
    optimizer = Adam(model.parameters(), lr=0.00020441990333108206)
    loss = nn.CrossEntropyLoss()
    
    if torch.cuda.is_available():
        cuda='cuda:'+str(gpu_ids[0])
        model = nn.DataParallel(model, device_ids=gpu_ids)
        loss.cuda()
    device = torch.device(cuda if torch.cuda.is_available() else 'cpu')
    model.to(device)
    return model, loss, optimizer, device

In [13]:
def train(device, model, loss, optimizer, train_dataset, validation_dataset, epochs, patience, path, verbose=0, batch_size=338):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    min_val_loss = np.Inf
    max_val_acc = np.NINF
    epochs_no_improve_loss = 0
    epochs_no_improve_acc = 0
    if verbose == 1:
        verbScheduler = True
    else:
        verbScheduler = False
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=80, cooldown=10, verbose=verbScheduler)
    training_generator = DataLoader(train_dataset, batch_size=batch_size)
    validation_generator = DataLoader(validation_dataset, batch_size=batch_size)
    model.train()
    for epoch in range(epochs):
        loss_train_epoch = []
        acc_train_epoch = []
        for i, (ramanSpectraTrain, labelTrain) in enumerate(training_generator):
            ramanSpectraTrain = ramanSpectraTrain.to(device)
            labelTrain = labelTrain.to(device)
        
            optimizer.zero_grad()
        
            output_train = model(ramanSpectraTrain)
            
            loss_train = loss(output_train, labelTrain)
            loss_train_epoch.append(loss_train.cpu().item())
            
            loss_train.backward()
            optimizer.step()
            
            output_label = torch.argmax(output_train, dim=1)
            acc_train = accuracy_score(labelTrain.cpu().detach().numpy(), output_label.cpu().detach().numpy())
            acc_train_epoch.append(acc_train)
        
        loss_train = mean(loss_train_epoch)
        acc_train = mean(acc_train_epoch)
        train_losses.append(loss_train)
        train_acc.append(acc_train)
        
        with torch.no_grad():
            loss_val_epoch = []
            acc_val_epoch = []
            for j, (ramanSpectraVal, labelVal) in enumerate(validation_generator):
                ramanSpectraVal = ramanSpectraVal.to(device)
                labelVal = labelVal.to(device)
                    
                output_val = model(ramanSpectraVal)
                    
                loss_val = loss(output_val, labelVal)
                loss_val_epoch.append(loss_val.cpu().item())
                
                val_label = torch.argmax(output_val, dim=1)
                acc_val = accuracy_score(labelVal.cpu().detach().numpy(), val_label.cpu().detach().numpy())
                acc_val_epoch.append(acc_val)
            
            loss_val = mean(loss_val_epoch)
            acc_val = mean(acc_val_epoch)
        val_losses.append(loss_val)
        val_acc.append(acc_val)
        scheduler.step(loss_val)
        if acc_val > max_val_acc:
            epochs_no_improve_acc = 0
            max_val_acc = acc_val
            torch.save({'model_state_dict' : model.state_dict(),
                       'optimizer_state_dict' : optimizer.state_dict(),
                       'train_loss' : train_losses,
                       'train_acc' : train_acc,
                       'val_loss' : val_losses,
                       'val_acc' : val_acc}, path)
        else:
            epochs_no_improve_acc += 1
        
        if loss_val < min_val_loss:
            epochs_no_improve_loss = 0
            min_val_loss = loss_val
            torch.save({'model_state_dict' : model.state_dict(),
                       'optimizer_state_dict' : optimizer.state_dict(),
                       'train_loss' : train_losses,
                       'train_acc' : train_acc,
                       'val_loss' : val_losses,
                       'val_acc' : val_acc}, path)
        else:
            epochs_no_improve_loss += 1
            
        if verbose == 1:
            print("Epoch {}:\t train loss : {}; train accuracy : {}; \n validation loss : {}; validation accuracy : {}".format(epoch+1, loss_train, acc_train, loss_val, acc_val))
            
        if epochs_no_improve_loss >= patience and epochs_no_improve_acc >= patience:
            print("Early stopping at epoch {}".format(epoch+1))
            break
    
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    train_losses = checkpoint['train_loss']
    train_acc = checkpoint['train_acc']
    val_losses = checkpoint['val_loss']
    val_acc = checkpoint['val_acc']
    
    if verbose == 1:
        print("----- Final result of the model ! -----")
        print("Train loss : {}; Train accuracy : {}; \n Validation loss : {}; Validation accuracy : {}".format(train_losses[-1], train_acc[-1], val_losses[-1], val_acc[-1]))
        
    return train_losses, val_losses, train_acc, val_acc

In [14]:
def visualizeTrain(modelName, train_loss, val_loss, train_acc, val_acc):
    plt.plot(train_acc, 'c-')
    plt.plot(train_loss, 'c--')
    plt.plot(val_acc, 'r-')
    plt.plot(val_loss, 'r--')
    plt.title(modelName)
    plt.ylabel('value')
    plt.xlabel('epoch')
    plt.legend(['trainning accuracy', 'training loss', 'validation accuracy', 'validation loss'], loc='upper left')
    plt.savefig('../train_results/'+modelName)

## Train!

In [15]:
num_epochs = 273
patience = 50
models_list = []
optimizers = []
loss_train_list = []
loss_val_list = []
acc_train_list = []
acc_val_list = []
start = time.time()
for i in range(len(train_dataset)):
    print("--- Let's train model {} ! ---".format(i+1))
    model, loss, optimizer, device = createCNN(gpus_list)
    path = "../saved_models/covid_RAW/"+str(i+1)+".pckl"
    train_loss, val_loss, train_acc, val_acc = train(device, model, loss, optimizer, train_dataset[i], validation_dataset[i], num_epochs, patience, path, verbose=1)
    #modelName = 'Model ' + str(i+1)
    #visualizeTrain(modelName, train_loss, val_loss, train_acc, val_acc)
    loss_train_list.append(train_loss)
    loss_val_list.append(val_loss)
    acc_train_list.append(train_acc)
    acc_val_list.append(val_acc)
    models_list.append(model)
    optimizers.append(optimizer)
end = time.time()

--- Let's train model 1 ! ---
Epoch 1:	 train loss : 0.9127732905389903; train accuracy : 0.6237105920341032; 
 validation loss : 0.8144422509313127; validation accuracy : 0.7333333333333333
Epoch 2:	 train loss : 0.7396704536950462; train accuracy : 0.8109232837055418; 
 validation loss : 0.7636383142179974; validation accuracy : 0.7875
Epoch 3:	 train loss : 0.7067314401292997; train accuracy : 0.842295066647579; 
 validation loss : 0.7482756314645039; validation accuracy : 0.8041666666666667
Epoch 4:	 train loss : 0.6839379570276491; train accuracy : 0.8664622701533371; 
 validation loss : 0.7043889664236868; validation accuracy : 0.8458333333333333
Epoch 5:	 train loss : 0.6669917595757693; train accuracy : 0.8834432859324298; 
 validation loss : 0.7362373754505993; validation accuracy : 0.8083333333333333
Epoch 6:	 train loss : 0.6501803635393667; train accuracy : 0.9001007070369663; 
 validation loss : 0.7081266940935269; validation accuracy : 0.8416666666666667
Epoch 7:	 train l

Epoch 54:	 train loss : 0.5653755565153494; train accuracy : 0.9857424484634473; 
 validation loss : 0.6172914761821893; validation accuracy : 0.9375
Epoch 55:	 train loss : 0.5652867710694145; train accuracy : 0.9858865997009607; 
 validation loss : 0.6490915313665597; validation accuracy : 0.8958333333333334
Epoch 56:	 train loss : 0.566217192671704; train accuracy : 0.984800991760514; 
 validation loss : 0.5973221995232386; validation accuracy : 0.9541666666666667
Epoch 57:	 train loss : 0.5715168694607639; train accuracy : 0.9794470159699689; 
 validation loss : 0.5925923064148128; validation accuracy : 0.9583333333333334
Epoch 58:	 train loss : 0.5653893725382054; train accuracy : 0.9859666284914423; 
 validation loss : 0.614841915603518; validation accuracy : 0.9375
Epoch 59:	 train loss : 0.5693992094118282; train accuracy : 0.9818185523636827; 
 validation loss : 0.6146968016767612; validation accuracy : 0.9333333333333333
Epoch 60:	 train loss : 0.5641996344979239; train accur

Epoch 107:	 train loss : 0.5596396420209336; train accuracy : 0.9915109833301521; 
 validation loss : 0.6074576168906768; validation accuracy : 0.9416666666666667
Epoch 108:	 train loss : 0.5599173023023318; train accuracy : 0.9915025330851943; 
 validation loss : 0.6103239255302854; validation accuracy : 0.9416666666666667
Epoch 109:	 train loss : 0.5597583933204872; train accuracy : 0.9915040243048928; 
 validation loss : 0.654022816895369; validation accuracy : 0.8958333333333334
Epoch 110:	 train loss : 0.5626151263992661; train accuracy : 0.9885538946681937; 
 validation loss : 0.6050395855515458; validation accuracy : 0.9458333333333333
Epoch 111:	 train loss : 0.5636996205363577; train accuracy : 0.987427526722657; 
 validation loss : 0.6123105096082582; validation accuracy : 0.9375
Epoch 112:	 train loss : 0.5590804277460508; train accuracy : 0.9922898970859578; 
 validation loss : 0.5934815796662353; validation accuracy : 0.9583333333333334
Epoch 113:	 train loss : 0.560071046

Epoch 159:	 train loss : 0.5576104254403764; train accuracy : 0.9936752401857861; 
 validation loss : 0.5894914732776508; validation accuracy : 0.9625
Epoch 160:	 train loss : 0.5598345088822153; train accuracy : 0.9914801647897181; 
 validation loss : 0.5787324696732973; validation accuracy : 0.975
Epoch 161:	 train loss : 0.5589632950381567; train accuracy : 0.9923823527072596; 
 validation loss : 0.5865104428266006; validation accuracy : 0.9666666666666667
Epoch 162:	 train loss : 0.5575042522923386; train accuracy : 0.9938392743526118; 
 validation loss : 0.6180044060272166; validation accuracy : 0.9291666666666667
Epoch 163:	 train loss : 0.5603477283977033; train accuracy : 0.9908891447159127; 
 validation loss : 0.6294212573994427; validation accuracy : 0.9208333333333333
Epoch 164:	 train loss : 0.5620361486596549; train accuracy : 0.9892304113380416; 
 validation loss : 0.6169256828032298; validation accuracy : 0.9333333333333333
Epoch 165:	 train loss : 0.5588172545294502; tr

Epoch 37:	 train loss : 0.569812571118144; train accuracy : 0.9815242107871991; 
 validation loss : 0.6107253472397869; validation accuracy : 0.9414225941422594
Epoch 38:	 train loss : 0.5681321680972203; train accuracy : 0.9830732054896373; 
 validation loss : 0.5915035171264559; validation accuracy : 0.9623430962343096
Epoch 39:	 train loss : 0.574008844980458; train accuracy : 0.9773054927352148; 
 validation loss : 0.6011137058316454; validation accuracy : 0.9497907949790795
Epoch 40:	 train loss : 0.5693369089183801; train accuracy : 0.981968834226587; 
 validation loss : 0.6061332634318014; validation accuracy : 0.9414225941422594
Epoch 41:	 train loss : 0.5650946803943973; train accuracy : 0.9862021747885622; 
 validation loss : 0.5907994303924375; validation accuracy : 0.9623430962343096
Epoch 42:	 train loss : 0.5658275441522916; train accuracy : 0.9856763840267666; 
 validation loss : 0.6239862409665978; validation accuracy : 0.9246861924686193
Epoch 43:	 train loss : 0.56947

...

Epoch 48:	 train loss : 0.56122955778078; train accuracy : 0.9900678459679668; 
 validation loss : 0.607037459783721; validation accuracy : 0.9414225941422594
Epoch 49:	 train loss : 0.5602379474190942; train accuracy : 0.9912079060689613; 
 validation loss : 0.6039542981341953; validation accuracy : 0.9456066945606695
Epoch 50:	 train loss : 0.5616494417296257; train accuracy : 0.9896805972923572; 
 validation loss : 0.6076060711603708; validation accuracy : 0.9456066945606695
Epoch 51:	 train loss : 0.5639560187580483; train accuracy : 0.9872858514823879; 
 validation loss : 0.6231898991461884; validation accuracy : 0.9288702928870293
Epoch 52:	 train loss : 0.5689551972537447; train accuracy : 0.9821462870596983; 
 validation loss : 0.6332736018010753; validation accuracy : 0.9205020920502092
Epoch 53:	 train loss : 0.5716856721652821; train accuracy : 0.9794479382880511; 
 validation loss : 0.6372606653454501; validation accuracy : 0.9121338912133892
Epoch 54:	 train loss : 0.56897

Epoch 99:	 train loss : 0.5733758882900503; train accuracy : 0.9775395768146473; 
 validation loss : 0.609632842388162; validation accuracy : 0.9414225941422594
Epoch 100:	 train loss : 0.5624822377354686; train accuracy : 0.9886892406827968; 
 validation loss : 0.5997530411695342; validation accuracy : 0.9497907949790795
Epoch 101:	 train loss : 0.5600555614758554; train accuracy : 0.9912233960159856; 
 validation loss : 0.607665143918228; validation accuracy : 0.9456066945606695
Epoch 102:	 train loss : 0.5592053085835201; train accuracy : 0.9921342049010192; 
 validation loss : 0.5998761000339276; validation accuracy : 0.9497907949790795
Epoch 103:	 train loss : 0.55720257032105; train accuracy : 0.9941200161095449; 
 validation loss : 0.6054197607165211; validation accuracy : 0.9456066945606695
Epoch 104:	 train loss : 0.5581088063963713; train accuracy : 0.9931813253198674; 
 validation loss : 0.6223349143880863; validation accuracy : 0.9288702928870293
Epoch 105:	 train loss : 0.

Epoch 3:	 train loss : 0.6980058870459758; train accuracy : 0.8523699618947304; 
 validation loss : 0.7015897077478839; validation accuracy : 0.8368200836820083
Epoch 4:	 train loss : 0.6763226690240826; train accuracy : 0.8743154992409926; 
 validation loss : 0.7012883216252606; validation accuracy : 0.8451882845188284
Epoch 5:	 train loss : 0.657467415948857; train accuracy : 0.8930140338920041; 
 validation loss : 0.6508128141518873; validation accuracy : 0.899581589958159
Epoch 6:	 train loss : 0.6426106709643272; train accuracy : 0.9077177112054277; 
 validation loss : 0.6802498989619925; validation accuracy : 0.8661087866108786
Epoch 7:	 train loss : 0.6325599197770262; train accuracy : 0.918485392979956; 
 validation loss : 0.6607386596132672; validation accuracy : 0.8786610878661087
Epoch 8:	 train loss : 0.621150001864056; train accuracy : 0.929901483936925; 
 validation loss : 0.6443098545690109; validation accuracy : 0.9037656903765691
Epoch 9:	 train loss : 0.61163604847830

Epoch 54:	 train loss : 0.5645397198567426; train accuracy : 0.9867811890083336; 
 validation loss : 0.6133883538441949; validation accuracy : 0.9372384937238494
Epoch 55:	 train loss : 0.562461423452596; train accuracy : 0.9887542984602993; 
 validation loss : 0.5891515505723947; validation accuracy : 0.9623430962343096
Epoch 56:	 train loss : 0.564025064298949; train accuracy : 0.9872053037578612; 
 validation loss : 0.6025457103854851; validation accuracy : 0.9497907949790795
Epoch 57:	 train loss : 0.5636443450357065; train accuracy : 0.9876021562006257; 
 validation loss : 0.5869307968900037; validation accuracy : 0.9665271966527197
Epoch 58:	 train loss : 0.5721822364280036; train accuracy : 0.9789742557080455; 
 validation loss : 0.6020602575586081; validation accuracy : 0.9497907949790795
Epoch 59:	 train loss : 0.5689801539381705; train accuracy : 0.982289104371263; 
 validation loss : 0.593145194311092; validation accuracy : 0.9581589958158996
Epoch 60:	 train loss : 0.561258

Epoch 105:	 train loss : 0.5593307292256456; train accuracy : 0.9920167911025745; 
 validation loss : 0.5949384535881522; validation accuracy : 0.9539748953974896
Epoch 106:	 train loss : 0.5571799216973219; train accuracy : 0.9941853836859877; 
 validation loss : 0.5893131155713132; validation accuracy : 0.9581589958158996
Epoch 107:	 train loss : 0.5582191912577451; train accuracy : 0.9931165773413055; 
 validation loss : 0.5798930822655968; validation accuracy : 0.9707112970711297
Epoch 108:	 train loss : 0.5704149405539147; train accuracy : 0.9806877536478825; 
 validation loss : 0.5941826110597157; validation accuracy : 0.9539748953974896
Epoch 109:	 train loss : 0.5636236876196727; train accuracy : 0.987608042380495; 
 validation loss : 0.6051918998409741; validation accuracy : 0.9456066945606695
Epoch 110:	 train loss : 0.5587185357161325; train accuracy : 0.9925958053223458; 
 validation loss : 0.604228877175267; validation accuracy : 0.9456066945606695
Epoch 111:	 train loss :

In [16]:
print("Training time :", (end-start)/3600)

Training time : 23.853519005576768


In [21]:
hT = (end-start)//3600
mT = ((end-start)%3600)//60
sT = (((end-start)%3600)%60)
print("------------------------------ Total time of training {} h {} m and {} s ------------------------------".format(hT, mT, sT))

------------------------------ Total time of training 23.0 h 51.0 m and 12.66842007637024 s ------------------------------


In [17]:
#for i in range(len(models_list)):
    #path = "../saved_models/covid_RAW/"+str(i)+".pckl"
    #torch.save({
            #'model': models_list[i].state_dict(),
            #'optimizer': optimizers[i].state_dict()}, path)

## Test

In [18]:
def testModel(model, test_set, device, batch_size=1):
    test_acc = []
    test_generator = DataLoader(test_set, batch_size=batch_size)
    model.eval()
    for i, (ramanSpectra, label) in enumerate(test_generator):
        ramanSpectra = ramanSpectra.to(device)
        label = label.to(device)
        
        labelPredict = model(ramanSpectra)
        labelPredict = torch.argmax(labelPredict, dim=1)
        
        acc = accuracy_score(label.cpu().detach().numpy(), labelPredict.cpu().detach().numpy())
        test_acc.append(acc)
    
    return test_acc

In [19]:
test_accs = []
for i in range(len(test_dataset)):
    print("--- Let's predict with model {} ! ---".format(i+1))
    acc = testModel(models_list[i], test_dataset[i], device)
    test_accs.append(acc)

for i in range(len(test_accs)):
    print("Model {} predict with {} of accuracy".format(i+1, mean(test_accs[i])))

--- Let's predict with model 1 ! ---
--- Let's predict with model 2 ! ---
--- Let's predict with model 3 ! ---
--- Let's predict with model 4 ! ---
--- Let's predict with model 5 ! ---
--- Let's predict with model 6 ! ---
--- Let's predict with model 7 ! ---
--- Let's predict with model 8 ! ---
--- Let's predict with model 9 ! ---
--- Let's predict with model 10 ! ---
--- Let's predict with model 11 ! ---
--- Let's predict with model 12 ! ---
--- Let's predict with model 13 ! ---
--- Let's predict with model 14 ! ---
--- Let's predict with model 15 ! ---
--- Let's predict with model 16 ! ---
--- Let's predict with model 17 ! ---
--- Let's predict with model 18 ! ---
--- Let's predict with model 19 ! ---
--- Let's predict with model 20 ! ---
--- Let's predict with model 21 ! ---
--- Let's predict with model 22 ! ---
--- Let's predict with model 23 ! ---
--- Let's predict with model 24 ! ---
--- Let's predict with model 25 ! ---
--- Let's predict with model 26 ! ---
--- Let's predict wit

In [20]:
total_acc = 0
for i in range(len(test_accs)):
    total_acc += mean(test_accs[i])
print("The mean accuracy is {}".format(total_acc/len(test_accs))) 

The mean accuracy is 0.8316799206409498
